## Flores Analysis

In [ ]:
import os
import json
from constants import LANGUAGE_MAP, LANGUAGE_TIER
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go

import jieba
import nagisa
from konlpy.tag import Kkma
import tltk
import deepcut
from tqdm import tqdm

### Loading FLORES BlockList

In [85]:
BLOCK_LIST = dict()
for language, code in LANGUAGE_MAP.items():
    BLOCK_LIST[language] = []
    if type(code) == list:
        for c in code:
            BLOCK_LIST[language] += [val.strip() for val in open(f"datasets/FLORES/{c}_twl/{c}_twl.txt").readlines()]
    else:
        BLOCK_LIST[language] = [val.strip() for val in open(f"datasets/FLORES/{code}_twl/{code}_twl.txt").readlines()]    

In [86]:
print(len(BLOCK_LIST["HI"]))

133


### Loading RTP-LX Dataset

In [87]:
RTP_LX = dict()
for language in LANGUAGE_MAP.keys():
    RTP_LX[language] = [json.loads(val) for val in open(f"datasets/RTP_LX/RTP_LX_{language}.json").readlines()]

RTP = dict()
for key, val in RTP_LX.items():
    RTP[key] = RTP_LX[key][:999]

LX = dict()
for key, val in RTP_LX.items():
    LX[key] = RTP_LX[key][999:]

### Calculating Block Rate

In [3]:
EVAL_TYPE = "Prompt"
# EVAL_TYPE = "BenignCompletion"

In [100]:
def calculate_block_rate(dataset, eval_type="Prompt"):
    results = dict()

    for language in dataset.keys():
        block = 0
        # print(language)
        if len(dataset[language]) > 0:
            for obj in tqdm(dataset[language], desc=language):
                try:
                    if "ZH" in language:
                        sentence = jieba.lcut(obj[eval_type], cut_all=True)
                    elif language == "JA":
                        sentence = nagisa.tagging(obj[eval_type]).words
                    elif language == "KO":
                        sentence = Kkma().morphs(obj[eval_type])
                    elif language == "TH":
                        sentence = tltk.nlp.pos_tag(obj[eval_type])
                        sentence = [word[0] for piece in sentence for word in piece]
                        # print(sentence)
                    else:
                        sentence = obj[eval_type].split(' ')
                        
                    for toxic in BLOCK_LIST[language]:
                        if toxic in sentence:
                            block += 1
                            # print(toxic)
                            break
                except:
                    print('Skipping')
            
            results[language] = round(block/len(dataset[language]), 2)
        else:
            print(f"Empty dataset: {language}")
            results[language] = 0
        print()
    
    return results

In [101]:
os.makedirs(f"results/{EVAL_TYPE.lower()}/flores_analysis", exist_ok=True)

In [102]:
block_rate = dict()
print("RTP-LX")
block_rate["RTP-LX"] = calculate_block_rate(RTP_LX, eval_type=EVAL_TYPE)
print("RTP")
block_rate["RTP"] = calculate_block_rate(RTP, eval_type=EVAL_TYPE)
print("LX")
block_rate["LX"] = calculate_block_rate(LX, eval_type=EVAL_TYPE)

with open(f"results/{EVAL_TYPE.lower()}/flores_analysis/block_rate.json", "w") as f:
    json.dump(block_rate, f, indent=4)

RTP-LX


NO-NB: 100%|██████████| 1099/1099 [00:00<00:00, 15182.34it/s]


CS: 100%|██████████| 1038/1038 [00:00<00:00, 2603.28it/s]


BCMS: 100%|██████████| 1057/1057 [00:00<00:00, 3348.58it/s]


RU: 100%|██████████| 1225/1225 [00:00<00:00, 3910.42it/s]


...

NL: 100%|██████████| 76/76 [00:00<00:00, 12107.53it/s]


AR: 100%|██████████| 51/51 [00:00<00:00, 5068.71it/s]


DA: 100%|██████████| 96/96 [00:00<00:00, 15251.44it/s]


SV: 100%|██████████| 50/50 [00:00<00:00, 25061.57it/s]


KO: 100%|██████████| 100/100 [00:15<00:00,  6.57it/s]

### Plotting

In [4]:
block_rate = pd.DataFrame(json.load(open(f"results/{EVAL_TYPE.lower()}/flores_analysis/block_rate.json")))

In [5]:
block_rate.head()

,RTP-LX,RTP,LX
NO-NB,0.28,0.30,0.09
CS,0.21,0.22,0.21
BCMS,0.25,0.27,0.05
RU,0.22,0.25,0.08
SW,0.20,0.21,0.04


In [6]:
block_rate["LX"]

NO-NB      0.09
CS         0.21
BCMS       0.05
RU         0.08
SW         0.04
PL         0.10
JA         0.11
FI         0.08
PT         0.21
UK         0.14
ZH-Hans    0.20
DE         0.03
ID         0.02
ZH-Hant    0.02
HE         0.05
TH         0.01
HI         0.02
FR         0.15
EN         0.00
TR         0.00
HU         0.05
IT         0.11
ES         0.09
NL         0.03
AR         0.06
DA         0.29
SV         0.00
KO         0.03
Name: LX, dtype: float64

In [7]:
round(block_rate["LX"].mean(), 3)*100

8.1

In [8]:
round(block_rate["LX"].std(), 3)*100

7.5

In [57]:
# languages = block_rate.index.tolist()
# languages = sorted(languages)
# languages.remove("EN")
# languages = ["EN"] + languages

# values1 = [block_rate["RTP"][val] for val in languages]
# values2 = [block_rate["LX"][val] for val in languages]
# values3 = [block_rate["RTP-LX"][val] for val in languages]

# # Width of each bar
# bar_width = 0.20

# # Positions for the bars
# x = np.arange(len(languages))

# plt.figure(figsize=(10, 6)) 
# # Plotting the bars for each group
# plt.bar(x - bar_width, values1, width=bar_width, edgecolor = 'black',label='Transcreated')
# plt.bar(x, values2, width=bar_width,edgecolor = 'black', label='Manual')
# plt.bar(x + bar_width, values3, width=bar_width, edgecolor = 'black',label='Full Corpus')

# # Adding labels and title
# plt.xlabel('Languages', fontdict={"size": 12, "fontfamily": "sans-serif"})
# plt.ylabel('Block Rate', fontdict={"size": 12, "fontfamily": "sans-serif"})
# plt.title('FLORES Toxicity-200 Block Rate (Toxic Prompt)', fontdict={"size": 14, "fontfamily": "sans-serif"})
# plt.xticks(x, languages, rotation=90)
# plt.legend(prop={"size": 10, "family": "sans-serif"})
# plt.tight_layout()

# plt.savefig(f"results/{EVAL_TYPE.lower()}/flores_analysis/block_rate.png", dpi=300)

# # Displaying the plot
# plt.show()


In [9]:
SUBSET_MAP = {
    "RTP": 'Transcreated',
    "LX": 'Manual',
    "RTP-LX": 'Full Corpus'
}

In [27]:
block_rate.head()

,RTP-LX,RTP,LX
NO-NB,0.28,0.30,0.09
CS,0.21,0.22,0.21
BCMS,0.25,0.27,0.05
RU,0.22,0.25,0.08
SW,0.20,0.21,0.04


In [30]:
block_rate = block_rate.sort_index()
block_rate.head()

,RTP-LX,RTP,LX
AR,0.14,0.15,0.06
BCMS,0.25,0.27,0.05
CS,0.21,0.22,0.21
DA,0.25,0.25,0.29
DE,0.13,0.13,0.03


In [50]:
# These will form the points on the circle
categories = block_rate.index.tolist()
categories.append(categories[0]) 

fig = go.Figure()
# increase figure size
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=1
    )
)

# these will form the rings inside
for col in block_rate.columns:
    values = block_rate[col].tolist()
    values.append(values[0])
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories,
        name=SUBSET_MAP[col],
        opacity=0.75
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0,0.6]
    )),
  showlegend=True
)

# set the legend at the bottom with 4 columns
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    xanchor="center",
    x=0.5,
    y=1.15,
))

# fig.update_layout(
#     title=dict(
#         text="FLORES Toxicity-200 Block Rate (Toxic Prompts)",
#         font=dict(size=14),
#         y=0.9,  # Adjust the vertical position of the title
#         x=0.5,  # Center the title horizontally
#         xanchor='center',
#         yanchor='top'
#     )
# )

# plt.tight_layout()

In [51]:
fig.write_image(f"results/{EVAL_TYPE.lower()}/flores_analysis/block_rate_polar_prompt.png", engine="kaleido")

### Notes
- Check RTP-LX subdivisions
- Confirm language codes (especially BCMS)